# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096804


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:29,  1.00s/it]

Rendering models:  10%|▉         | 3/31 [00:06<00:56,  2.02s/it]

Rendering models:  13%|█▎        | 4/31 [00:06<00:40,  1.49s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:31,  1.21s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:22<00:02,  1.35it/s]

Rendering models:  94%|█████████▎| 29/31 [00:23<00:01,  1.38it/s]

Rendering models:  97%|█████████▋| 30/31 [00:23<00:00,  1.71it/s]

Rendering models: 100%|██████████| 31/31 [00:24<00:00,  1.58it/s]

ganta.sasank                          0.001291
Jnursssmith5263                       0.001520
not-logged-in-56a50e105eb68df25a17    0.001852
ChelseaB98                            0.004950
TateK                                 0.000614
Quarkgc                               0.000395
jnarayanbvg                           0.009214
awright5                              0.017779
Borkdoge123                           0.198177
BokGlobule                            0.026189
koungeo8                              0.005717
tingard                               0.000212
Theaker                               0.000892
not-logged-in-24db7619b192ada33b8d    0.000413
aidenr                                0.000734
acapirala                             0.001770
kferreira1                            0.000307
alexmnelson18                         0.000285
Lavadude                              0.205541
clars915                              0.000249
not-logged-in-2027f1f6703c19b00bd8    0.001259
brigi.baumann

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())